# Equilibrium Line Altitude 

This notebook shows how you can compute the Equilbrium Line Altitude (ELA), the height where the mass balance equals 0, with OGGM. 

As the ELA in OGGM only 
depends on the glacier its sensitivity to the climate and the climate itself (https://docs.oggm.org/en/stable/mass-balance.html), there is no need to do a full model
run to collect these values. This is also the reason that the ELA is no longer a part of the model run output, like was the case up to OGGM version 1.4, but is instead as of v1.6 a diagnostic variable that can be computed seperately.

In [ ]:
from oggm import cfg, utils, workflow, global_tasks
from oggm.core import massbalance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cfg.initialize(logging_level='WARNING')

In [ ]:
# Where to store the data 
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-ela', reset=True)
# Which glaciers?
rgi_ids = ['RGI60-11.00897', 'RGI60-11.00001']
# Here we change some settings to the settings of the preprocessed directory that will be used. 
cfg.PARAMS['climate_qc_months'] = 0
cfg.PARAMS['hydro_month_nh']=1
cfg.PARAMS['hydro_month_sh']=1
# We start from prepro level 3 with all data ready
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L3-L5_files/CRU/elev_bands/qc0/pcp2.5/match_geod_pergla/'
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=3, prepro_base_url=base_url, prepro_border=80)

Here we compute and compile the ELA. 

By the default the file is stored in the working directory and named 'ELA.hdf'. In stead of storing the data as .hdf, you can choose to save the data in a csv file, by setting the csv keyword to True. The location where the file is saved can be changed by specifying the path keyword.

In [ ]:
ELA = global_tasks.compile_ela(gdirs, ys=2000, ye=2019) 

Here the ELA data is plotted: 

In [ ]:
plt.plot(ELA)
plt.xlim([2000,2019])
plt.xlabel('year')
plt.ylabel('ELA [m]')
plt.legend(ELA.columns.values)

When looking at many glaciers it might be usefull to also plot the mean or average of the data.
Here is the mean ELA and the glacier area weigthed ELA average is plotted:

In [ ]:
Area1 = pd.read_csv(gdirs[0].get_filepath('elevation_band_flowline'))['area'].sum()
Area2 = pd.read_csv(gdirs[1].get_filepath('elevation_band_flowline'))['area'].sum()

plt.plot(ELA.mean(axis=1), label='mean')
plt.plot(np.arange(2000,2020), np.average(ELA, axis=1, weights=[Area1,Area2]), label='average')

plt.xlim([2000,2019])
plt.xlabel('year')
plt.ylabel('ELA [m]')
plt.legend()

How would to ela have looked like if the climate had been 1 degree warmer? Lets have a look at one of the glaciers:

In [ ]:
global_tasks.compile_ela(gdirs[0], ys=2000, ye=2019, temperature_bias=1, filesuffix='_t1')
ELA2 = pd.read_hdf(cfg.PATHS['working_dir'] + '/ELA_t1.hdf')

In [ ]:
ELA.columns[0]

In [ ]:
plt.plot(ELA[ELA.columns[0]], label='default climate')
plt.plot(ELA2, label=('default climate +1℃'))

plt.title(ELA.columns[0])
plt.xlim([2000,2019])
plt.xlabel('year CE')
plt.ylabel('ELA [m]')
plt.legend()

By using the precipitation_factor keyword, you can change the precipitation. Feel free to try that out yourself.

Lets look for 1 glacier at a longer timeseries: 

In [ ]:
global_tasks.compile_ela(gdirs[0], ys=1901, ye=2019, filesuffix='_1901_2019', csv=True)
ELA3 = pd.read_csv(cfg.PATHS['working_dir'] + '/ELA_1901_2019.csv', index_col=0)

The ELA can have a high year-to-year variability. Therefore we plot in addition to the regular ELA timeseries, the 5-year moving average. 

In [ ]:
plt.plot(ELA3)
plt.plot(ELA3.rolling(5).mean(), lw=2, color='k')
plt.xlim([1900,2020])
plt.xlabel('year CE')
plt.ylabel('ELA [m]')

In case you're only intrested in specific years there is an option to just compute the ELA in those years. There is actually no need to save this data. Therefore we now just compute the ELA. 

In [ ]:
ELA4 = massbalance.compute_ela(gdirs[0], years=[2000, 1950, 1980, 1960,])
ELA4 

See in this case you can use any year as long as you have climate available for that year. However for plotting purposes it might be worth to sort the data, otherwise the following happens ;)

In [ ]:
ELA4.plot()
plt.xlim([1950,2000])
plt.xlabel('year CE')
plt.ylabel('ELA [m]')

In [ ]:
plt.plot(ELA4.sort_index(ascending=True))
plt.xlim([1950,2000])
plt.xlabel('year CE')
plt.ylabel('ELA [m]')

By now we have addressed most of the keyword agruments of the compile_ela and compute_ela functions. To use different climate than the default climate, you can make use of the 'climate_filename' and 'climate_input_filesuffix' keywords. In case you're not familiar with those yet, please check out the run_with_gcm notebook.